# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}(r_{ij} - u_i - a_j)^2 + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$

In [1]:
const name = "UserItemBiases"
const residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

## Alternating Least Squares Algorithm
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function thread_range(n)
    tid = Threads.threadid()
    nt = Threads.nthreads()
    d, r = divrem(n, nt)
    from = (tid - 1) * d + min(r, tid - 1) + 1
    to = from + d - 1 + (tid ≤ r ? 1 : 0)
    from:to
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)

    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [5]:
function train_model(training, stop_criteria, λ_u, λ_a, λ_wu, λ_wa)
    @info "training model with parameters [$λ_u, $λ_a, $λ_wu, $λ_wa]"
    users, items, ratings = training.user, training.item, training.rating
    weights =
        safe_exp.(get_counts("training"), log(λ_wu)) .*
        safe_exp.(get_counts("training"; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), maximum(users))
    a = zeros(eltype(λ_a), maximum(items))

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

In [6]:
function make_prediction(users, items, u, a)
    r = zeros(eltype(u), length(users))
    u_mean = mean(u)
    a_mean = mean(a)
    for i = 1:length(r)
        if users[i] > length(u)
            r[i] += mean(u)
        else
            r[i] += u[users[i]]
        end
        if items[i] > length(a)
            r[i] += mean(a)
        else
            r[i] += a[items[i]]
        end
    end
    r
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ...)
    pred_score = make_prediction(validation.user, validation.item, u, a)
    weights = get_weights("validation", "inverse")
    mse(validation.rating, pred_score, weights)
end;

In [9]:
# Find the best regularization hyperparameters
res = optimize(
    validation_mse,
    fill(0.0f0, 4),
    BFGS(linesearch = BackTracking()),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220520 04:51:37 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.12 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (35.22 ns/it)


Iter     Function value   Gradient norm 
     0     1.820281e+00     5.464972e-02
 * Current step size: 1.0
 * time: 0.024962902069091797
 * g(x): Float32[-0.0050086514, -1.0955863f-6, 0.019451264, 0.05464972]
 * ~inv(H): Float32[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220520 04:52:06 training model with parameters [1.0050212, 1.0000011, 0.9807367, 0.94681674]
[ Info: 20220520 04:52:36 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0050212,1.0050212,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000011,0.0,1.0000011,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9807367,0.0,0.0,0.9807367,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.94681674,0.0,0.0,0.0,0.94681674)]


     1     1.818474e+00     1.522718e-02
 * Current step size: 1.0
 * time: 74.55671286582947
 * g(x): Float32[-0.00035009012, -1.7580297f-6, 0.015227176, -0.010049476]
 * ~inv(H): Float32[1.0020685 2.7653282f-6 -0.02710825 -0.0034922624; 2.7653282f-6 1.0 -1.4911706f-5 -2.599969f-5; -0.027108252 -1.4911705f-5 1.1793545 0.22169183; -0.003492276 -2.599969f-5 0.22169183 0.8299486]
 * x: Float32[0.0050086514, 1.0955863f-6, -0.019451264, -0.05464972]


[ Info: 20220520 04:53:18 training model with parameters [1.0057536, 1.0000029, 0.9654209, 0.951528]
[ Info: 20220520 04:53:47 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0057536,1.0057536,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000029,0.0,1.0000029,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9654209,0.0,0.0,0.9654209,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.951528,0.0,0.0,0.0,0.951528)]


...

    14     1.814476e+00     3.677324e-05
 * Current step size: 1.0
 * time: 999.4623157978058
 * g(x): Float32[-7.3892475f-6, 3.6773235f-5, 3.2598882f-6, 1.372408f-5]
 * ~inv(H): Float32[654.7855 30.33923 44.20008 44.417217; 30.339228 2.4138737 2.1257844 2.0780945; 44.20008 2.1257844 18.144285 -0.0804076; 44.41723 2.078095 -0.080407575 4.0392094]
 * x: Float32[-0.33894736, -0.017446, -0.43877035, -0.016496988]


[ Info: 20220520 05:08:43 training model with parameters [0.7146386, 0.9828035, 0.64495164, 0.98383176]
[ Info: 20220520 05:09:12 training model with parameters [0.7135785, 0.9827544, 0.64489025, 0.983735]
[ Info: 20220520 05:09:41 training model with parameters [0.71262574, 0.98271024, 0.644835, 0.983648]
[ Info: 20220520 05:10:09 training model with parameters [0.71253055, 0.9827058, 0.64482945, 0.9836393]
[ Info: 20220520 05:10:38 training model with parameters [0.71252525, 0.98270553, 0.64482915, 0.9836388]
[ Info: 20220520 05:11:07 training model with parameters [0.7125222, 0.9827054, 0.644829, 0.9836385]
[ Info: 20220520 05:11:35 training model with parameters [0.71252096, 0.98270535, 0.6448289, 0.9836384]
[ Info: 20220520 05:12:04 training model with parameters [0.71252036, 0.9827053, 0.64482886, 0.9836384]
[ Info: 20220520 05:12:33 training model with parameters [0.7125201, 0.9827053, 0.64482886, 0.98363835]
[ Info: 20220520 05:13:01 training model with parameters [0.71252006, 

    15     1.814476e+00     3.677324e-05
 * Current step size: 3.7724752e-5
 * time: 1329.2332727909088
 * g(x): Float32[-7.3962333f-6, 3.6773243f-5, 3.264613f-6, 1.358048f-5]
 * ~inv(H): Float32[654.7855 30.33923 44.20008 44.417217; 30.339228 2.4138737 2.1257844 2.0780945; 44.20008 2.1257844 18.144285 -0.0804076; 44.41723 2.078095 -0.080407575 4.0392094]
 * x: Float32[-0.33894724, -0.017445996, -0.43877035, -0.01649698]


[ Info: 20220520 05:14:13 training model with parameters [0.7146464, 0.98280406, 0.64495176, 0.98383266]
[ Info: 20220520 05:14:42 training model with parameters [0.7135824, 0.98275465, 0.6448903, 0.9837355]
[ Info: 20220520 05:15:10 training model with parameters [0.7126262, 0.98271024, 0.644835, 0.98364806]
[ Info: 20220520 05:15:39 training model with parameters [0.7125307, 0.9827058, 0.64482945, 0.9836393]
[ Info: 20220520 05:16:08 training model with parameters [0.71252537, 0.98270553, 0.64482915, 0.9836388]
[ Info: 20220520 05:16:36 training model with parameters [0.71252227, 0.9827054, 0.644829, 0.9836385]
[ Info: 20220520 05:17:05 training model with parameters [0.712521, 0.98270535, 0.6448289, 0.98363847]
[ Info: 20220520 05:17:34 training model with parameters [0.7125205, 0.9827053, 0.64482886, 0.9836384]
[ Info: 20220520 05:18:02 training model with parameters [0.71252024, 0.9827053, 0.64482886, 0.98363835]
[ Info: 20220520 05:18:31 training model with parameters [0.7125201,

    16     1.814476e+00     3.677323e-05
 * Current step size: 3.7724752e-5
 * time: 1658.5070049762726
 * g(x): Float32[-7.4009163f-6, 3.677323f-5, 3.2764353f-6, 1.3772849f-5]
 * ~inv(H): Float32[348.81418 10.903069 69.03034 4.8342795; 10.903065 1.4510566 2.379642 0.1389899; 69.03035 2.3796422 22.572678 0.33078313; 4.834296 0.13899067 0.3307831 0.13609947]
 * x: Float32[-0.33894712, -0.017445993, -0.43877035, -0.016496973]


In [10]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220520 05:19:42 The optimal λ is Float32[0.7125201, 0.9827053, 0.64482886, 0.98363835], found in 35 function calls


In [12]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ...);

[ Info: 20220520 05:19:42 training model with parameters [0.7125201, 0.9827053, 0.64482886, 0.98363835]


## Inference

In [13]:
model(users, items) = make_prediction(users, items, u, a);

In [14]:
write_predictions(model; residual_alphas = residual_alphas);

[ Info: 20220520 05:20:19 training set weighted-loss: RMSE 1.2923702 MAE 0.9603634 R2 0.4864248
[ Info: 20220520 05:20:20 validation set weighted-loss: RMSE 1.3469614 MAE 1.0024881 R2 0.39467883


In [15]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));